In [ ]:
!pip install pydub

In [ ]:
import requests
#MeitY: 64392f96daac500b55c543cd

In [ ]:
url = 'https://meity-auth.ulcacontrib.org/ulca/apis/v0/model/getModelsPipeline'

In [ ]:
headers = {
    'Content-Type':'application/json',
    'userID' : 'c0b5cfa7bd9a4e32832c45d3ecdadd8f',
    'ulcaApiKey':'137e95f6aa-4e8c-4eec-b019-ed3c7a976290'
}

In [ ]:
payload = {
    'pipelineTasks': [
        {
            'taskType':'asr',
        "config": {
                "language": {
                    "sourceLanguage": "sa"
                }
        }
        }
        ],
    'pipelineRequestConfig':
     {
         #'pipelineId':'643930aa521a4b1ba0f4c41d',
         'pipelineId':'64392f96daac500b55c543cd'
         }
}

In [ ]:
response = requests.post(url, json = payload, headers = headers)

In [ ]:
if response.status_code!=200:
  print({'status_code':response.status_code, 'message':"Error in asr request"})
else:
  print("It works. Go forward")

It works. Go forward


In [ ]:
response

<Response [200]>

In [ ]:
response_data = response.json()

In [ ]:
response_data

{'languages': [{'sourceLanguage': 'sa', 'targetLanguageList': ['sa']}],
 'pipelineResponseConfig': [{'taskType': 'asr',
   'config': [{'serviceId': 'ai4bharat/conformer-multilingual-indo_aryan-gpu--t4',
     'modelId': '6411742856e9de23f65b5422',
     'language': {'sourceLanguage': 'sa', 'sourceScriptCode': 'Deva'},
     'domain': ['general']}]}],
 'feedbackUrl': 'https://dhruva-api.bhashini.gov.in/services/feedback/submit',
 'pipelineInferenceAPIEndPoint': {'callbackUrl': 'https://dhruva-api.bhashini.gov.in/services/inference/pipeline',
  'inferenceApiKey': {'name': 'Authorization',
   'value': 'YbdeanQUDxKbRgQIZnW4PrB-7xrpLR3OvDn0eiy6xjWHuJBW-kVxF1v00zEHylQg'},
  'isMultilingualEnabled': True,
  'isSyncApi': True},
 'pipelineInferenceSocketEndPoint': {'callbackUrl': 'wss://dhruva-api.bhashini.gov.in',
  'inferenceApiKey': {'name': 'Authorization',
   'value': 'YbdeanQUDxKbRgQIZnW4PrB-7xrpLR3OvDn0eiy6xjWHuJBW-kVxF1v00zEHylQg'},
  'isMultilingualEnabled': True,
  'isSyncApi': True}}

In [ ]:
service_id = response_data['pipelineResponseConfig'][0]['config'][0]['serviceId']
service_id

'ai4bharat/conformer-multilingual-indo_aryan-gpu--t4'

In [ ]:
callback_url = response_data['pipelineInferenceAPIEndPoint']['callbackUrl']
callback_url

'https://dhruva-api.bhashini.gov.in/services/inference/pipeline'

In [ ]:
headers2 = {
    'Content-Type':'application/json',
    response_data['pipelineInferenceAPIEndPoint']['inferenceApiKey']['name']:response_data['pipelineInferenceAPIEndPoint']['inferenceApiKey']['value']
    }

In [ ]:
headers2

{'Content-Type': 'application/json',
 'Authorization': 'YbdeanQUDxKbRgQIZnW4PrB-7xrpLR3OvDn0eiy6xjWHuJBW-kVxF1v00zEHylQg'}

In [ ]:
import pydub
from pydub import AudioSegment
audio_file_path ='/content/Recording.wav'
given_audio = AudioSegment.from_wav(audio_file_path)
given_audio = given_audio.set_frame_rate(16000)
given_audio.export("temp.wav",format="wav", codec="pcm_s16le")

<_io.BufferedRandom name='temp.wav'>

In [ ]:
import base64
with open("/content/temp.wav", "rb") as wav_file:
      encoded_string = base64.b64encode(wav_file.read())
  #Encode the file.
encoded_string = str(encoded_string,'ascii','ignore')

import base64
audio_file_path ='/content/Recording.wav'
files = {'audio':open(audio_file_path,'rb')}
def encode_file_to_base64(file_path):
    with open(file_path, 'rb') as file:
        file_content = file.read()
        base64_encoded = base64.b64encode(file_content).decode('utf-8')
    return base64_encoded
base64_audio_content = encode_file_to_base64(audio_file_path)

In [ ]:
compute_payload = {
    'pipelineTasks':[
        {
        'taskType':'asr',
        'config':{'language':{'sourceLanguage':'sa'},
        'serviceId':service_id,
        'audioFormat':'wav',
        "samplingRate": 16000
        }
        }
    ],
    'inputData':{
                 'audio':[
                     {'audioContent':encoded_string}
                 ]}
}

In [ ]:
response_2 = requests.post(callback_url, headers = headers2, json=compute_payload)
response_2

<Response [200]>

In [ ]:
if response_2.status_code!=200:
  print({'status_code':response_2.status_code,
         'message':'Error in asr request',
         'reponse_text':response.text
         })
else:
  source_value = response_2.json()['pipelineResponse'][0]['output'][0]['source']
  print(source_value)

पत्रः तेषाम् मध्ये राजसम्बन्धि विरूपः अपि एकः विरूपः बुध्या किञ्चित् वक्रः सः सर्वेषाम् विडम्बरम् करोति सः गुरुकुले सर्वदा नियमान्


In [ ]:
import requests

def bhashini_translate(text: str, user_id: str, api_key: str, from_code: str = "sa", to_code: str = "en") -> dict:
    """Translates text from source language to target language using the Bhashini API.

    Args:
        text (str): The text to translate.
        from_code (str): Source language code. Default is 'en' (English).
        to_code (str): Target language code. Default is 'te' (Telugu).
        user_id (str): User ID for the API.
        api_key (str): API key for authentication.

    Returns:
        dict: A dictionary with the status code, message, and translated text or error info.
    """
    # Setup the initial request to get model configurations
    url = 'https://meity-auth.ulcacontrib.org/ulca/apis/v0/model/getModelsPipeline'
    headers = {
        "Content-Type": "application/json",
        "userID": user_id,
        "ulcaApiKey": api_key
    }
    payload = {
        "pipelineTasks": [{"taskType": "translation", "config": {"language": {"sourceLanguage": from_code, "targetLanguage": to_code}}}],
        "pipelineRequestConfig": {"pipelineId": "64392f96daac500b55c543cd"}
    }
    response = requests.post(url, json=payload, headers=headers)

    if response.status_code != 200:
        return {"status_code": response.status_code, "message": "Error in translation request", "translated_content": None}

    # Process the response to setup the translation execution
    response_data = response.json()
    service_id = response_data["pipelineResponseConfig"][0]["config"][0]["serviceId"]
    callback_url = response_data["pipelineInferenceAPIEndPoint"]["callbackUrl"]
    headers2 = {
        "Content-Type": "application/json",
        response_data["pipelineInferenceAPIEndPoint"]["inferenceApiKey"]["name"]: response_data["pipelineInferenceAPIEndPoint"]["inferenceApiKey"]["value"]
    }
    compute_payload = {
        "pipelineTasks": [{"taskType": "translation", "config": {"language": {"sourceLanguage": from_code, "targetLanguage": to_code}, "serviceId": service_id}}],
        "inputData": {"input": [{"source": text}], "audio": [{"audioContent": None}]}
    }

    # Execute the translation
    compute_response = requests.post(callback_url, json=compute_payload, headers=headers2)
    if compute_response.status_code != 200:
        return {"status_code": compute_response.status_code, "message": "Error in translation", "translated_content": None}

    compute_response_data = compute_response.json()
    translated_content = compute_response_data["pipelineResponse"][0]["output"][0]["target"]

    return {"status_code": 200, "message": "Translation successful", "translated_content": translated_content}

In [ ]:
print(bhashini_translate(source_value,'c0b5cfa7bd9a4e32832c45d3ecdadd8f','137e95f6aa-4e8c-4eec-b019-ed3c7a976290'))

{'status_code': 200, 'message': 'Translation successful', 'translated_content': 'The letter is a distortion of the royal relationship between them; it is also a distortion of the law; it is a distortion of everything.'}


In [ ]:
def make_list(text):
    delimiter =["॥","।"]
    for delimiters in delimiter:
      text = text.replace(delimiters,'\n')
    #print(text)
    new_list = text.split('\n')
    #print(new_list)
    filtered_list = [item for item in new_list if item.strip()]
    #print(filtered_list)
    new_filtered_list =[item.replace('|','') for item in filtered_list]
    return new_filtered_list

new_list = make_list('यस्य कृत्यं न जानन्ति मन्त्र वा मन्त्रितं परे। कृतमेवास्य जानन्ति स वै पण्डित उच्यते ॥')
print(new_list)

['यस्य कृत्यं न जानन्ति मन्त्र वा मन्त्रितं परे', ' कृतमेवास्य जानन्ति स वै पण्डित उच्यते ']
